# Demo 1D Flint

This notebook shows how to process 1D relaxation decays using Flint.

### Importing Flint

To use Flint, we need to import it into our Python environment:

In [1]:
from pyflint.pyflint import Flint, NMRsignal
from pyflint.make_syntetic_data import time_distribution_signal_decay
import numpy as np

### Generating syntetic data

For this demo, we will be working with a synthetic dataset generated by the function `time_distribution_signal_decay`, which is included with Flint. This dataset consists of a bimodal 1D T2 relaxation decay, and can be generated using the following cell.

Note that the `amplitudes`, `centers`, and `widths` arrays can be extended to generate more populations. The generated signal is saved in the `signal_with_noise` variable, while the T2 distribution is saved in the `t2_distribution_intensity` variable. We will be using these variables to demonstrate how to process 1D relaxation decays using Flint.

In [26]:
signal_time_axis, signal_with_noise, ILT_time_axis, t2_distribution_intensity = time_distribution_signal_decay(
        signal_num_points=64,
        signal_time_lim=[1e-4, 6],
        kernel_name="T2",
        normalized_noise=0.5e-4,
        t_dimension=500,
        t_axis_lim=[1e-4, 3],
        amplitudes=[0.5, 0.5],
        centers=[20e-3, 200e-3],
        widths=[5e-3, 50e-3],
        plot=True
    )

### Processing data

Now that we have loaded our syntetic NMR data, we can use Flint to process it. We will start by converting the data to an NMRsignal class.

In [13]:
# load simulated NMR signal
nmr_signal = NMRsignal.load_from_data(signal_with_noise, signal_time_axis)

Note that NMR data can be also loaded from a text file, for example:

In [10]:
nmr_signal = NMRsignal.load_from_1d_txtfile('./example_data/T2decay.dat')

Now we will set the parameters needed to perform the ILT:
- `t1_range`: The range of the first dimension relaxation time values to be used in the inversion. It is a numpy array with two elements, representing the minimum and maximum t1 values. In this demo, we use a range of [0.2, 3] seconds.
- `kernel_name`: The name of the kernel used. In this case "T2" kernel.
- `kernel_dimensions`: The dimensions of the kernel matrix used in the inversion. It is a list with two elements, representing the number of points in the t1 and t2 axes, respectively. In this demo, we use a kernel with 500 points in the t1 axis and 1 point in the t2 axis, as we are only considering 1D data.
- `regularization_parameter`: The value of the regularization parameter used in the inversion. It is a small positive number that helps to stabilize the inversion and avoid overfitting. In this demo, we use a regularization parameter of 1e-6.

In [25]:
# ILT parameters setting
t1_range = np.array([1e-3, 3e0])
kernel_dimensions = [256, 1]
regularization_parameter = 0.8e-2

# Calling Flint method
flint = Flint(nmr_signal, kernel_dimensions, "T2", regularization_parameter, t1_range)
flint.solve_flint()
flint.plot()

Lipschitz constant found: 13694.605695420007
